# Burger & Windhager et al. (in preparation, 2022)

In [ ]:
from pathlib import Path

import napari
import numpy as np
import pandas as pd
from scipy.ndimage import affine_transform
from skimage.transform import EuclideanTransform
from tifffile import imread, imwrite

## Image extraction

Run [steinbock.sh](steinbock.sh)

In [ ]:
volume_dir = Path("volumes")
volume_dir.mkdir(exist_ok=True)
image_info = pd.read_csv("images.csv")
for source_file, group in image_info.groupby("source_file"):
    img = np.array(
        [
            imread(Path("img") / img_file_name)
            for img_file_name in group["image"].sort_values()
        ]
    )
    imwrite(volume_dir / f"{Path(source_file).stem}.tiff", data=img)

In [ ]:
# scale = (0.6, 1, 1)
# viewer = napari.Viewer()
# channel_names = pd.read_csv("panel.csv")["name"].dropna().tolist()
# viewer.open("volumes/20220221_MB_3DAblation_BCOrganoidInEPON.tiff", channel_axis=1, name=channel_names, colormap="gray", scale=scale, visible=False);
# viewer.open("volumes/20220303_MB_3DAblation_BCOrganoidsInEPON.tiff", channel_axis=1, name=channel_names, colormap="gray", scale=sacle, visible=False);

## Image registration

In [ ]:
img1 = imread("volumes/20220221_MB_3DAblation_BCOrganoidInEPON.tiff")
img2 = imread("volumes/20220303_MB_3DAblation_BCOrganoidsInEPON.tiff")
napping_transform = np.load("napping_euclidean_transform_layer101_to_layer100.npy")

In [ ]:
transform = np.eye(4)
transform[0, 3] = img1.shape[0]
transform[[1, 2], 3] = napping_transform[[1, 0], 2]
transform[[1, 2], [2, 1]] = napping_transform[[1, 0], [0, 1]]
transform = EuclideanTransform(matrix=transform)

In [ ]:
img1_min, img1_max = np.array([0, 0, 0]), np.array(img1.shape)[[0, 2, 3]]
img2_min, img2_max = transform([[0, 0, 0], np.array(img2.shape)[[0, 2, 3]]])
img_min = np.minimum(img1_min, img2_min)
img_max = np.maximum(img1_max, img2_max)
img_shape = np.ceil(img_max).astype(int) - np.floor(img_min).astype(int)
img_shape = (img_shape[0], img1.shape[1], img_shape[1], img_shape[2])

img1_transformed = np.zeros(img_shape, dtype=img1.dtype)
img1_transformed
for c in range(img1.shape[1]):
    img1_transformed[:, c, :, :] = affine_transform(
        img1[:, c, :, :],
        np.eye(3),
        offset=img_min,
        output=img1_transformed[:, c, :, :],
        order=1,  # linear interpolation
    )

img2_transformed = np.zeros(img_shape, dtype=img2.dtype)
for c in range(img2.shape[1]):
    img2_transformed[:, c, :, :] = affine_transform(
        img2[:, c, :, :],
        np.linalg.inv(transform.params)[:3, :3],
        offset=transform.inverse(img_min)[0],
        output=img2_transformed[:, c, :, :],
        order=1,  # linear interpolation
    )
    
img = np.maximum(img1_transformed, img2_transformed)
img = img[15:145, :, 195:280, 167:252]
imwrite("img.tiff", data=img)

In [ ]:
# scale = (0.6, 1, 1)
# viewer = napari.Viewer(ndisplay=3)
# channel_names = pd.read_csv("panel.csv")["name"].dropna().tolist()
# viewer.open("img.tiff", channel_axis=1, name=channel_names, colormap="gray", scale=scale, visible=False);

## Image segmentation

In [ ]:
# import sys
# !{sys.executable} -m pip install cellpose

In [ ]:
# import os

# from cellpose.models import Cellpose

# os.environ["CELLPOSE_LOCAL_MODELS_PATH"] = "/mnt/data/cellpose/"

In [ ]:
# model = Cellpose(gpu=True, model_type="nuclei")
# img = imread("img.tiff")
# nucl_img = np.amax(img[:, [3, 4, 10], :, :], axis=1)  # H3K9ac, Ki-67, H3K4me2
# mask, _, _, _ = model.eval(
#     nucl_img,
#     channels=[0, 0],
#     z_axis=0,
#     diameter=12,
#     do_3D=True,
#     anisotropy=0.6,
#     min_size=50,
#     progress=True,
# )
# imwrite("mask.tiff", data=mask.astype(np.uint16))

In [ ]:
# scale = (0.6, 1, 1)
# viewer = napari.Viewer(ndisplay=3)
# channel_names = pd.read_csv("panel.csv")["name"].dropna().tolist()
# viewer.open("img.tiff", channel_axis=1, name=channel_names, colormap="gray", scale=scale, visible=False);
# viewer.open("mask.tiff", layer_type="labels", name="Cell mask", scale=scale, visible=False);